In [52]:
base_path = "/Users/lorenapacheco/Concordia/Masters/"
coverage_data_folder = base_path + "coverage_data/"

bug_fixes_added_modified = {
    "fastjson": {
        "2351": {
            "fix_commit" : "d30069235d846b910d8371171aa855e4eea3a97b",
            "modified_lines": {
                "JSONLexerBase": ["1397"],
                "ASMDeserializerFactory": ["384"]
            },
            "added_tests": [
                "Issue2351"
            ]
        },
        "3280": {
            "fix_commit" : "92fd0cb92cd876e01384418b6dbcae557a8a630c",
            "modified_lines": {
                "JavaBeanDeserializer": ["1460"]
            },
            "added_tests": [
            ]
        }
    }
}

bug_fix_deleted = {
    "fastjson": {
        "3119": {
            "buggy_commit" : "307f6637b9f8502a081468443c2b3c85cea5e545",
            "deleted_lines": {
                "ASMSerializerFactory": ["151"]
            }
        },
        "3280": {
            "buggy_commit" : "cc7dfab3dde888b81577fb18c0eb223761b5e31f",
            "deleted_lines": {
                "JavaBeanDeserializer": ["1460"]
            }
        }
    }
}

In [53]:
import os
import glob
import json

def json_file_to_dict(file):
    data = {}
    with open(os.path.join(file), 'r') as fp:
        data = json.load(fp)
    fp.close()
    return data


def dict_to_json_file(file, dic, folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    with open(os.path.join(folder, file+'.json'), 'w') as fp:
        json.dump(dic, fp, sort_keys=True, indent=4)
    fp.close()

def run_checkout(project_path, buggy_commit):
    os.chdir(project_path)
    checkout_command = "git checkout  --quiet " + buggy_commit
    os.system(checkout_command)


def find_file(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

def read_file_lines(file_name, project_path):
    path = find_file(file_name, project_path)
    if path:
        with open(path, 'r', encoding='utf-8', errors='ignore') as file:
            lines =file.readlines()
        file.close()
        return lines
    return []


def test_covers_the_line(test_data, file_name, line):
    for test_file in test_data["covered_lines"].keys():
        if test_file.endswith(file_name):
            return line in test_data["covered_lines"][test_file]
    return False


def get_tests_that_cover_the_line(coverage_details_folder, file, line, tests_to_skip):
    tests_that_cover_the_line = []
    for test_coverage_file in glob.glob(coverage_details_folder + "*.json"):
        test_name = os.path.basename(test_coverage_file).replace('.json', '')
        if test_name == "coverage_details" or test_name in tests_to_skip:
            continue
        test_data = json_file_to_dict(test_coverage_file)
        if test_covers_the_line(test_data, file, line):
            tests_that_cover_the_line.append(test_name)
    return tests_that_cover_the_line

def get_tests_that_cover_all_the_lines(coverage_details_folder, modified_lines, tests_to_skip):
    tests_that_cover_all_the_lines = []
    if not os.path.exists(coverage_details_folder):
        print("WARN: Coverage folder does not exist for the commit")
        return []
    for test_coverage_file in glob.glob(coverage_details_folder + "*.json"):
        test_name = os.path.basename(test_coverage_file).replace('.json', '')
        if test_name == "coverage_details" or test_name in tests_to_skip:
            continue
        test_data = json_file_to_dict(test_coverage_file)
        test_cover_all_the_lines = True
        for file in modified_lines.keys():
            lines = modified_lines[file]
            for line in lines:
                if not test_covers_the_line(test_data, file, line):
                    test_cover_all_the_lines = False
        if test_cover_all_the_lines:
            tests_that_cover_all_the_lines.append(test_name)
    return tests_that_cover_all_the_lines

print("!!!!!!! DELETED")
for project in bug_fix_deleted.keys():
    print("------ "+ project + " ------")

    for bug_report_id in bug_fix_deleted[project].keys():
        print("** "+ bug_report_id + " **")
        buggy_commit = bug_fix_deleted[project][bug_report_id]["buggy_commit"]
        deleted_lines = bug_fix_deleted[project][bug_report_id]["deleted_lines"]
        coverage_details_folder = coverage_data_folder + project + "/" + buggy_commit + "/details/"
        result = get_tests_that_cover_all_the_lines(coverage_details_folder, deleted_lines, [])
        print(len(result))
        print(result)


print("!!!!!!! ADDED/MODIFIED")
for project in bug_fixes_added_modified.keys():
    print("------ "+ project + " ------")

    for bug_report_id in bug_fixes_added_modified[project].keys():
        print("** "+ bug_report_id + " **")
        bugfix_commit = bug_fixes_added_modified[project][bug_report_id]["fix_commit"]
        modified_lines = bug_fixes_added_modified[project][bug_report_id]["modified_lines"]
        added_tests = bug_fixes_added_modified[project][bug_report_id]["added_tests"]
        coverage_details_folder = coverage_data_folder + project + "/" + bugfix_commit + "/details/"
        result = get_tests_that_cover_all_the_lines(coverage_details_folder, modified_lines, added_tests)
        print(len(result))
        print(result)
        #for file in modified_lines.keys():
        #    lines = modified_lines[file]
        #    for line in lines:
        #        tests_that_cover_the_line = get_tests_that_cover_the_line(coverage_details_folder, file, line, added_tests)
        #        print(file + " - " + line + " - " )
        #        print(tests_that_cover_the_line)


!!!!!!! DELETED
------ fastjson ------
** 3119 **
696
['BeanToArrayAutoTypeTest', 'BeanToArrayAutoTypeTest2', 'Issue1647', 'ClassLevelFeatureConfigTest3', 'Issue1493', 'NameFilterTest_byte', 'Issue2179', 'WuqiTest', 'Issue1939', 'Bug_for_smoothrat', 'JSONFieldTest2', 'WriteAsArray_double_private', 'Issue1256', 'WriteAsArray_Object_public', 'Bug_for_issue_349', 'Issue87_treeset', 'JSONObjectTest_readObject', 'Issue1344', 'JSONPath_field_access_filter_notNull', 'PublicFieldStringTest', 'Issue1503', 'DoubleTest', 'ZonedDateTimeTest', 'JSONFieldTest', 'Issue1063_date', 'TestSpecial_entity', 'Issue2952', 'Issue1368', 'OffseTimeTest', 'DateFieldTest3', 'PublicFieldLongTest', 'TransientTest', 'BooleanFieldSerializerTest', 'RuntimeExceptionTest', 'Bug_for_SpitFire_3', 'SpecialStringTest', 'LongFieldTest_3', 'WriteNonStringValueAsStringTestDoubleField', 'MultiPointTest', 'bug_for_pengsong0302', 'SerializeWriterTest_19', 'SerializeWriterTest_BrowserSecure', 'Bug_for_apollo0317', 'Bug_for_Double2